# **Treinamento Básico de SmartWorks IoT**
## **Parte 2 - MQTT**

O Message Queuing Telemetry Transport ou MQTT é um protocolo de rede leve de `publish/subscription` que transporta mensagens entre dispositivos. O protocolo geralmente é executado sobre TCP / IP; no entanto, qualquer protocolo de rede que forneça conexões bidirecionais ordenadas sem perdas pode suportar MQTT.

MQTT é centrado em dados, enquanto HTTP é centrado em documentos. HTTP é o protocolo de solicitação-resposta para computação cliente-servidor e nem sempre otimizado para dispositivos móveis. Os principais benefícios sólidos do MQTT nesses termos são leveza (o MQTT transfere dados como uma matriz de bytes) e o modelo de `publish/subscription`, o que o torna perfeito para dispositivos com recursos limitados e ajuda a economizar bateria e banda. 

## **Coletando dados do dispositivo**
Assim como na primeira parte desse treinamento, utilizaremos os dados do nosso computador para simular um dispositivo conectado à internet.

In [1]:
import modules.computer as pc
import json

pc.update_metrics()
print(json.dumps(pc.computer_data, indent=2))

{
  "cpu": "39.2",
  "memory": "68.54",
  "memory_available": "50.36",
  "memory_percentage": "26.5",
  "memory_used": "18.19",
  "memory_free": "50.36",
  "disk": "1024.21",
  "disk_used": "828.28",
  "disk_free": "195.93",
  "disk_percentage": "80.9"
}


## **Identificando nosso dispositivo**
A forma como identificaremos nosso dispositivo para comunicações MQTT é exatamente igual à forma como identificamos no HTTP.

In [2]:
space = "trn-monitora"
collection = "computadores"
thing_id = "01FDZGA4YDZCQW233YP15T0TZW"

## Athenticação

Diferentemente de HTTP, para comunicarmos via MQTT precisamos apenas saber o username e o password do dispositivo definidos na plataforma.

In [3]:
username = "pc_teste@trn-monitora"
password = "monitora123"

## Tópicos

Quando estamos falando de `publish/subscription` precisamos também entender o conceito de tópicos. Ao publicar uma mensagem, nós sempre estaremos publicando-a a um determinado tópico, apenas receberam essa mensagem, os `subscriptors` que realizarem a subscrição em naquele tópico no momento do envio da mensagem.

O subscritor receberá apenas as mensagens enviadas a partir do momento da subscrição/assinatura, ou seja, ele não tem acesso ao histórico de envio. 

Um mesmo tópico pode ter vários subscriptors assim como vários publishers (apesar de o segundo caso não ser indicado)

Vamos definir a estrutura de tópicos que utilizaremos

In [4]:
base_topic = "{}/collections/{}/things/{}".format(space, collection, thing_id)
print(base_topic)

trn-monitora/collections/computadores/things/01FDZGA4YDZCQW233YP15T0TZW


Além disso, também precisaremos também difinir o endereço do nosso host.

In [5]:
mqtt_host = "mqtt.swx.altairone.com"

## Publicando um conjunto de dados

Para subscrever-se a uma propriedade, assim como qualquer outro endpoint, utilizaremos a biblioteca `paho-mqtt`para nos ajudar com a subscrição

In [30]:
import paho.mqtt.client as mqtt

client = mqtt.Client()
client.username_pw_set(username = username, password = password)
client.connect(mqtt_host, port = 1883, keepalive = 3600, bind_address="")

0

recebemos um 0, o que significa que a conexão foi bem sucedida. a conexão deverá durar 1 hora, caso ela finalize-se, basta executar o código acima novamente.

Para publicar a um tópico, utilizamos o formato "set/base_topic/endpoint". Então, para publicar ao tópico data, o tópico ficará assim.

In [20]:
data_pub = "set/{}/data".format(base_topic)
print(data_pub)

set/trn-monitora/collections/computadores/things/01FDZGA4YDZCQW233YP15T0TZW/data


In [24]:
import json

pc.update_metrics()
client.publish(topic = data_pub, payload = json.dumps(pc.computer_data))

## Subscrevendo ao tópico de dados

Como haviamos mencionado anteriormente, todo tópico pode ter `publishers` e `subscriptors`. Quando vemos o novo data point ser adicionado à plataforma, a plataforma está atuando como um subscriptor e guardando as informações que ela lê no feed daquele tópico.

Nós podemos entretando, subscrever a esse tópico de forma que podemos interceptar a informação e realizar a leitura ao mesmo tempo que a plataforma.

Para que possamos visualizar a mensagem recebida, precisamos primeiro criar uma função que será chamada toda vez que o nosso subscriptor receber uma mensagem.

In [25]:
def on_message(client, userdata, message):
    print("message received " ,str(message.payload.decode("utf-8")))

Uma vez que a função foi criada, podemos realizar a subscrição. 

In [26]:
import time

client.subscribe(topic=data_pub)

client.on_message = on_message
client.loop_start()
time.sleep(20)
client.loop_stop()

message received  {"cpu": "99.2", "memory": "68.54", "memory_available": "42.25", "memory_percentage": "38.4", "memory_used": "26.29", "memory_free": "42.25", "disk": "1024.21", "disk_used": "827.92", "disk_free": "196.29", "disk_percentage": "80.8"}
message received  {"cpu": "99.2", "memory": "68.54", "memory_available": "42.25", "memory_percentage": "38.4", "memory_used": "26.29", "memory_free": "42.25", "disk": "1024.21", "disk_used": "827.92", "disk_free": "196.29", "disk_percentage": "80.8"}


## Propriedades, Ações e Eventos


Para publicar ao tópico de propriedades utilizaremos a mesma estrutura de tópico utilizada para o envio de dados, mudando apenas o endpoint.

In [27]:
cpu_property_pub = "set/{}/properties/cpu_usage".format(base_topic)
print(cpu_property_pub)

set/trn-monitora/collections/computadores/things/01FDZGA4YDZCQW233YP15T0TZW/properties/cpu_usage


O payload deve seguir os mesmos padrões deve ser concordante com a REST, caso contrário a propriedade não será atualizada

In [32]:
pc.update_metrics()

payload = {
    "cpu_usage": float(pc.computer_data["cpu"])
}

client.publish(topic = cpu_property_pub, payload = json.dumps(payload))
print(payload)

{'cpu_usage': 47.4}


É possível também subscrever-se ao mesmo tópico, assim como fizemos com o data.

Para Propriedades, Ações e Eventos, a plataforma disponibiliza outra estrutura de subcrição. Para acessa-la, basta substituir o `set` por `status` no início do tópico.

In [33]:
cpu_property_sub = "status/{}/properties/cpu_usage".format(base_topic)
print(cpu_property_sub)

status/trn-monitora/collections/computadores/things/01FDZGA4YDZCQW233YP15T0TZW/properties/cpu_usage


In [34]:
client.subscribe(topic=cpu_property_sub)

client.on_message = on_message
client.loop_start()
time.sleep(20)
client.loop_stop()

message received  {"cpu_usage":85}
message received  {"cpu_usage":90}


Nessa estrutura, uma nova mensagem é publicada pela plataforma toda vez que uma propriedade/ação/evento é atualizada. Vejamos qual o resultado ao acessar uma ação.

In [35]:
cpu_action_sub = "status/{}/actions/update_properties".format(base_topic)

client.subscribe(topic=cpu_action_sub)

client.on_message = on_message
client.loop_start()
time.sleep(20)
client.loop_stop()

message received  {"update_properties":{"href":"/spaces/trn-monitora/collections/computadores/things/01FDZGA4YDZCQW233YP15T0TZW/actions/update_properties/01FEEVX9ZWDY85ZCHZCPBHH7N2","status":"pending","timeRequested":"2021-08-31T19:45:22+0000"}}
